<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1SQciZw0CMjRbCCJ815bQiTbn_tGly_yN?usp=sharing)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

## ⚡ Semantic Kernel: Versatile AI Orchestration Framework

Semantic Kernel is an open-source framework for **integrating large language models (LLMs)** into traditional applications, enabling seamless AI orchestration and automation. 🚀

### 🔑 **Key Features**:
- 🧩 Modular kernel architecture for streamlined service and plugin management.
- 🌐 Built-in connectors for OpenAI, Azure OpenAI, and Hugging Face.
- ✏️ Supports prompt engineering, dynamic task planning, and function orchestration.
- 📈 Scalable and customizable for AI-driven applications.


### **Install Semantic Kernel**

In [ ]:
!pip install semantic-kernel
from semantic_kernel import __version__
print(f"Semantic Kernel version: {__version__}")

### **Initial Configuration for the Notebook**


In [2]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### **Configuring the Kernel**


In [3]:
from semantic_kernel import Kernel

kernel = Kernel()

### **Using OpenAI Configuration**


In [4]:
from google.colab import userdata
import os

OPENAI_API_KEY= userdata.get('OPENAI_API_KEY')
OPENAI_CHAT_MODEL_ID='gpt-4o'

os.environ['GLOBAL_LLM_SERVICE'] = 'OpenAI'
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_CHAT_MODEL_ID'] = 'gpt-4o'
os.environ['OPENAI_TEXT_MODEL_ID'] = 'gpt-4'

##**Creating an OpenAI Assistant with Semantic Kernel**


### **Import Libraries**

In [5]:
import asyncio
import os
import re

from openai import OpenAI
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

### **Display Introductory Message**


In [6]:
def display_intro_message():
    print(
        """
    Chat with an AI assistant backed by a Semantic Kernel OpenAIAssistantAgent.

    To start: you can upload files to the assistant using the command (brackets included):

    [upload code_interpreter | file_search file_path]

    where `code_interpreter` or `file_search` is the purpose of the file and
    `file_path` is the path to the file. For example:

    [upload code_interpreter file.txt]

    This will upload file.txt to the assistant for use with the code interpreter tool.

    Type "exit" to exit the chat.
    """
    )


### **Parse Upload Command**


In [7]:
def parse_upload_command(user_input: str):
    """Parse the user input for an upload command."""
    match = re.search(r"\[upload\s+(code_interpreter|file_search)\s+(.+)\]", user_input)
    if match:
        return match.group(1), match.group(2)
    return None, None

### **Handle File Upload**


In [8]:
async def handle_file_upload(assistant_agent: OpenAIAssistantAgent, purpose: str, file_path: str):
    """Handle the file upload command."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    file_id = await assistant_agent.add_file(file_path, purpose="assistants")
    print(f"File uploaded: {file_id}")

    if purpose == "code_interpreter":
        await enable_code_interpreter(assistant_agent, file_id)
    elif purpose == "file_search":
        await enable_file_search(assistant_agent, file_id)

### **Enable Code Interpreter**


In [9]:
async def enable_code_interpreter(assistant_agent: OpenAIAssistantAgent, file_id: str):
    """Enable the file for code interpreter."""
    assistant_agent.code_interpreter_file_ids.append(file_id)
    tools = [{"type": "file_search"}, {"type": "code_interpreter"}]
    tool_resources = {"code_interpreter": {"file_ids": assistant_agent.code_interpreter_file_ids}}
    await assistant_agent.modify_assistant(
        assistant_id=assistant_agent.assistant.id, tools=tools, tool_resources=tool_resources
    )
    print("File enabled for code interpreter.")

### **Enable File Search**


In [10]:
async def enable_file_search(assistant_agent: OpenAIAssistantAgent, file_id: str):
    """Enable the file for file search."""
    if assistant_agent.vector_store_id is not None:
        await assistant_agent.client.beta.vector_stores.files.create(
            vector_store_id=assistant_agent.vector_store_id, file_id=file_id
        )
        assistant_agent.file_search_file_ids.append(file_id)
    else:
        vector_store = await assistant_agent.create_vector_store(file_ids=file_id)
        assistant_agent.file_search_file_ids.append(file_id)
        assistant_agent.vector_store_id = vector_store.id
        tools = [{"type": "file_search"}, {"type": "code_interpreter"}]
        tool_resources = {"file_search": {"vector_store_ids": [vector_store.id]}}
        await assistant_agent.modify_assistant(
            assistant_id=assistant_agent.assistant.id, tools=tools, tool_resources=tool_resources
        )
    print("File enabled for file search.")

### **Cleanup Resources**


In [11]:
async def cleanup_resources(assistant_agent: OpenAIAssistantAgent):
    """Cleanup the resources used by the assistant."""
    if assistant_agent.vector_store_id:
        await assistant_agent.delete_vector_store(assistant_agent.vector_store_id)
    for file_id in assistant_agent.code_interpreter_file_ids:
        await assistant_agent.delete_file(file_id)
    for file_id in assistant_agent.file_search_file_ids:
        await assistant_agent.delete_file(file_id)
    await assistant_agent.delete()

In [12]:
code = '''def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n-1)

num = int(input("Enter a number: "))
print(f"The factorial of {num} is {factorial(num)}")'''

with open('code.txt', 'w') as file:
    file.write(code)

print("Code saved to code.txt")

Code saved to code.txt


### **Main Chat Assistant Function**


In [13]:
async def main():
    assistant_agent = None
    try:
        display_intro_message()

        # Create the OpenAI Assistant Agent
        assistant_agent = await OpenAIAssistantAgent.create(
            service_id="AIAssistant",
            description="An AI assistant that helps with everyday tasks.",
            instructions="Help the user with their task.",
            enable_code_interpreter=True,
            enable_file_search=True,
            api_key=OPENAI_API_KEY,
            model_id=OPENAI_CHAT_MODEL_ID,
        )

        # Define an agent group chat, which drives the conversation
        # We add messages to the chat and then invoke the agent to respond.
        chat = AgentGroupChat()

        while True:
            try:
                user_input = input("User:> ")
            except (KeyboardInterrupt, EOFError):
                print("\n\nExiting chat...")
                break

            if user_input.strip().lower() == "exit":
                print("\n\nExiting chat...")
                break

            purpose, file_path = parse_upload_command(user_input)
            if purpose and file_path:
                await handle_file_upload(assistant_agent, purpose, file_path)
                continue

            await chat.add_chat_message(message=ChatMessageContent(role=AuthorRole.USER, content=user_input))
            async for content in chat.invoke(agent=assistant_agent):
                print(f"Assistant:> # {content.role} - {content.name or '*'}: '{content.content}'")
    finally:
        if assistant_agent:
            await cleanup_resources(assistant_agent)

await main()



    Chat with an AI assistant backed by a Semantic Kernel OpenAIAssistantAgent.

    To start: you can upload files to the assistant using the command (brackets included):

    [upload code_interpreter | file_search file_path]

    where `code_interpreter` or `file_search` is the purpose of the file and
    `file_path` is the path to the file. For example:

    [upload code_interpreter file.txt]

    This will upload file.txt to the assistant for use with the code interpreter tool.

    Type "exit" to exit the chat.
    
User:> hi
Assistant:> # AuthorRole.ASSISTANT - agent_ICoLFKUrNORUePvN: 'Hello! How can I assist you today?'
User:> exit


Exiting chat...


In [14]:
import asyncio
import openai
from typing import Annotated
from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.kernel import Kernel

###**Define Menu Plugin**



In [15]:
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"

###**Helper Method to Invoke Agent**


In [16]:

async def invoke_agent(agent: OpenAIAssistantAgent, thread_id: str, input: str) -> None:
    """Invoke the agent with the user input."""
    await agent.add_chat_message(thread_id=thread_id, message=ChatMessageContent(role=AuthorRole.USER, content=input))

    print(f"# {AuthorRole.USER}: '{input}'")

    async for content in agent.invoke(thread_id=thread_id):
        if content.role != AuthorRole.TOOL:
            print(f"# {content.role}: {content.content}")


###**Main Function to Create and Invoke Agent**


In [ ]:
async def main():
    # Create the instance of the Kernel
    kernel = Kernel()

    # Add the sample plugin to the kernel
    kernel.add_plugin(plugin=MenuPlugin(), plugin_name="menu")

    # Create the OpenAI Assistant Agent
    service_id = "agent"
    if use_azure_openai:
        agent = await AzureAssistantAgent.create(
            kernel=kernel, service_id=service_id, name=HOST_NAME, instructions=HOST_INSTRUCTIONS
        )
    else:
        agent = await OpenAIAssistantAgent.create(
            kernel=kernel, service_id=service_id, name=HOST_NAME, instructions=HOST_INSTRUCTIONS
        )

    thread_id = await agent.create_thread()

    try:
        await invoke_agent(agent, thread_id=thread_id, input="Hello")
        await invoke_agent(agent, thread_id=thread_id, input="What is the special soup?")
        await invoke_agent(agent, thread_id=thread_id, input="What is the special drink?")
        await invoke_agent(agent, thread_id=thread_id, input="Thank you")
    finally:
        await agent.delete_thread(thread_id)
        await agent.delete()

await main()